In [1]:
!pip install langchain langchain-openai


In [2]:
!pip install langchain-openai

In [3]:
!pip install --upgrade langchain-openai langchain

In [4]:

!pip install langchain-community

In [28]:
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:5 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
100 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy In

In [29]:
import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [30]:
!ollama pull llama3.2

In [31]:
from langchain.agents import initialize_agent, AgentType
from langchain.agents.agent import AgentExecutor
from langchain.tools import Tool
from langchain.chat_models import ChatOpenAI


In [32]:
import json
from typing import List, Dict
from langchain_core.documents import Document
from langchain_core.tools import tool as FunctionTool
from langchain_core.runnables import Runnable
from langchain.memory import ChatMessageHistory, ConversationBufferMemory
from langchain.memory import ConversationBufferMemory

from langchain_openai import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.vectorstores.utils import DistanceStrategy
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.text_splitter import CharacterTextSplitter
from langchain.indexes import VectorstoreIndexCreator


In [33]:
!pip install faiss-cpu

In [34]:
from langchain.llms import Ollama

In [35]:
import os

llm = Ollama(model="llama3.2", temperature=0.5)

embed_model = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")


documents = [
    Document(page_content="California employment law requires minimum $16/hour wage.", metadata={"type": "legal", "location": "California"}),
    Document(page_content="Startup technical roles typically take 45-60 days to fill.", metadata={"type": "metrics", "role": "technical"}),
    Document(page_content="Standard interview process: Phone screen -> Technical assessment -> Onsite -> Offer", metadata={"type": "process"})
]
index = FAISS.from_documents(documents, embedding=embed_model)
retriever = index.as_retriever()



In [36]:
memory = ConversationBufferMemory(token_limit=1000)

In [37]:
from langchain.tools import tool

@tool
def generate_job_description(role: str, experience_level: str, company_values: List[str]) -> str:
    """Generate a job description for a given role and experience level with company values."""
    prompt = f"""Create a job description for: {role} ({experience_level} level)
    Company values: {', '.join(company_values)}
    Include: Responsibilities, Requirements, and Benefits sections"""
    return str(llm.invoke(prompt).content)

@tool
def get_salary_benchmark(role: str, location: str, stage: str) -> Dict:
    """Fetch salary benchmark data (50th and 75th percentile) for the given role and location."""
    salary_data = {
        "CTO": {"50th": 150000, "75th": 180000},
        "Product Manager": {"50th": 120000, "75th": 140000}
    }
    return {
        "role": role,
        "location": location,
        "data": salary_data.get(role, {"50th": 100000, "75th": 120000})
    }

@tool
def create_interview_plan(role: str, level: str) -> Dict:
    """Create an interview process plan for the given role and level."""
    return {
        "role": role,
        "level": level,
        "process": [
            "Initial phone screen (30 mins)",
            "Technical assessment (1 hour)",
            "Team interview (2 hours)",
            "Culture fit interview (1 hour)",
            "Reference checks"
        ],
        "estimated_time": "3-4 weeks"
    }

@tool
def check_compliance(plan: Dict, location: str) -> Dict:
    """Check legal compliance of the hiring plan in the given location."""
    compliance_rules = {
        "California": ["Minimum wage $16/hr", "Anti-discrimination laws", "Mandatory harassment training"],
        "New York": ["Minimum wage $15/hr", "Salary history ban", "Paid family leave"]
    }
    return {
        "location": location,
        "requirements": compliance_rules.get(location, ["Standard US employment laws apply"]),
        "issues": []
    }


In [38]:
from langchain_core.tools import tool as FunctionTool


In [39]:

tools = [
    FunctionTool("generate_jd", generate_job_description),
    FunctionTool("get_salary", get_salary_benchmark),
    FunctionTool("create_interview", create_interview_plan),
    FunctionTool("check_compliance", check_compliance),
]


In [16]:
# tools_list = [generate_job_description, get_salary_benchmark, create_interview_plan, check_compliance]

# agent = initialize_agent(
#     tools=tools_list,
#     llm=llm,
#     agent=AgentType.OPENAI_FUNCTIONS,
#     memory=memory,
#     verbose=True
# )


<ipython-input-16-1e2ffb0c3f93>:3: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(


In [40]:


tools = [
    FunctionTool("generate_jd", generate_job_description),
    FunctionTool("get_salary", get_salary_benchmark),
    FunctionTool("create_interview", create_interview_plan),
    FunctionTool("check_compliance", check_compliance),
]


agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    memory=memory,
    verbose=True
)


In [41]:

def create_hiring_plan(company_info: Dict):
    needs_analysis = agent.invoke(
        f"Analyze hiring needs for {company_info['name']} "
        f"in {company_info['industry']} with {company_info['team_size']} employees"
    )

    role_priority = agent.invoke(
        "Based on this startup context, recommend role hiring priority:\n"
        f"{str(needs_analysis)}\nConsider their funding stage: {company_info['stage']}"
    )

    plan = {}
    for role in ["CTO", "Product Manager"]:
        plan[role] = {
            "job_description": generate_job_description(role, "senior", company_info["values"]),
            "salary_range": get_salary_benchmark(role, company_info["location"], company_info["stage"]),
            "interview_process": create_interview_plan(role, "senior")
        }

    compliance = check_compliance(plan, company_info["location"])
    return {
        "company": company_info["name"],
        "hiring_priority": str(role_priority),
        "detailed_plan": plan,
        "compliance_check": compliance
    }


# if __name__ == "__main__":
#     company_profile = {
#         "name": "TechNova",
#         "industry": "AI",
#         "team_size": 15,
#         "location": "California",
#         "stage": "Series A",
#         "values": ["innovation", "diversity", "agility"]
#     }

#     print("Generating hiring plan...")
#     try:
#         plan = create_hiring_plan(company_profile)
#         print("\nGenerated Hiring Plan:")
#         print(json.dumps(plan, indent=2))
#     except Exception as e:
#         print(f"Error generating plan: {e}")

#     print("\nAsk HR questions (type 'exit' to quit):")
#     while True:
#         try:
#             user_input = input("\nHR Question: ")
#             if user_input.lower() == 'exit':
#                 break
#             response = agent.invoke(user_input)
#             print(f"\nAssistant: {response}")
#         except Exception as e:
#             print(f"Error processing question: {e}")


In [42]:
import subprocess

def ollama_run(prompt: str) -> str:
    """Run an Ollama model via CLI and get the output."""
    result = subprocess.run(
        ['ollama', 'run', 'llama3.2', prompt],  # or your desired model name
        capture_output=True,
        text=True
    )
    if result.returncode != 0:
        raise RuntimeError(f"Ollama run failed: {result.stderr}")
    return result.stdout.strip()

if __name__ == "__main__":
    company_profile = {
        "name": "TechNova",
        "industry": "AI",
        "team_size": 15,
        "location": "California",
        "stage": "Series A",
        "values": ["innovation", "diversity", "agility"]
    }

    print("Generating hiring plan...")
    try:
        plan = create_hiring_plan(company_profile)
        print("\nGenerated Hiring Plan:")
        print(json.dumps(plan, indent=2))
    except Exception as e:
        print(f"Error generating plan: {e}")

    print("\nAsk HR questions (type 'exit' to quit):")
    while True:
        user_input = input("\nHR Question: ")
        if user_input.lower() == 'exit':
            break
        try:
            response = ollama_run(user_input)
            print(f"\nAssistant: {response}")
        except Exception as e:
            print(f"Error processing question: {e}")


Generating hiring plan...


> Entering new AgentExecutor chain...
I'd be happy to help analyze the hiring needs for TechNova's AI team.

**Current Team Structure:**
To get started, I'll need some information about the current team structure. Can you please provide me with the following details:

1. Number of employees currently working in the AI department
2. Job titles and roles of each employee (e.g., Data Scientist, Engineer, Researcher, etc.)
3. Key skills and areas of expertise that are currently represented by the team

**Hiring Needs:**
Based on our conversation, I'll provide some general suggestions for hiring needs. Please feel free to adjust or add to these based on your specific requirements:

1. **Data Scientist:** With 15 employees, it's likely that the team is working on various projects and datasets. A Data Scientist can help with data analysis, modeling, and visualization.
2. **Machine Learning Engineer:** As AI technology advances, the need for skilled engineers who ca

KeyboardInterrupt: Interrupted by user